In [7]:
import os
import math

In [8]:
#!pip install graphviz
from graphviz import Digraph

In [9]:
def get_label(longname):
    filename = os.path.basename(longname)
    return filename if len(filename) <= 10 else filename[:7] + "..."

def get_rel_paths(here_path, root_path):
    here_path = os.path.abspath(here_path)
    root_path = os.path.abspath(root_path)
    if here_path == root_path:
        return ('..', '.')
    rel_to_here = os.path.relpath(here_path, root_path)
    parent_dir = os.path.dirname(here_path)
    if parent_dir == root_path:
        return ('.', rel_to_here)
    rel_to_parent = os.path.basename(parent_dir)
    return (rel_to_parent, rel_to_here)

def remove_excluded_subdirs(dir_list):
    """
    For os.path.walk(), we can restrict the branches that get traversed by changing
    the list which walk() returns as 'subdirs'.  Remember that when we are editing
    that list, we are actually changing the memory representation inside the walk()
    generator, so the semantics are a little tricky.  For example, we can't simultaneously
    loop over the list and edit it.  I'll give you a working function for this task
    to avoid confusion.
    
    This function removes 'reveal.js', '.git', and anything starting with '_' from the list.
    """
    more_names_to_remove = [elt for elt in dir_list if elt.startswith('_')][:]
    for nm in ['reveal.js', '.git'] + more_names_to_remove:
        if nm in dir_list:
            dir_list.remove(nm)

def show_this_leaf(nm):
    if nm[0] == '.':
        return False
    if nm[0] == '_':
        return False
    return True

class Node():
    def __init__(self, parent_node, name):
        """
        parent_node should be the Node instance of the parent.
        
        name is a long name, like a full relative path.  It needs to be
        unique for all nodes in the tree.
        
        Note how we make a connection to the parent node when this node is created.
        """
        self.parent = parent_node
        if self.parent is not None:
            self.parent._add_kid(self)
        self.name = name
        self.label = get_label(self.name)
        self.kids = []
        self.descendant_count = 0
    def _add_kid(self, kid_node):
        self.kids.append(kid_node)
        self.descendant_count += 1
    def add_descendant_to_all_ancestors(self):
        self.descendant_count += 1
        if self.parent:
            self.parent.add_descendant_to_all_ancestors()

    def write_node(self, indent=0):
        shape = 'rectangle' if not self.kids else 'ellipse'
        color = 'lightgray'
        if self.name.endswith(('.png', '.jpg', 'svg')):
            color = 'blue'
        print(f'{indent*" "}"{self.name}" [label="{self.label}", shape={shape}, style=filled, fillcolor={color}];')
    def traverse_node_defs(self, indent=0):
        """
        Write the DOT code that defines this Node and all its descendants.
        
        Here and in traverse_edge_defs(), 'indent' just helps with formatting
        when writing out the DOT code.
        """
        self.write_node(indent=indent)
        for kid in self.kids:
            kid.traverse_node_defs(indent+4)
    def write_incoming_edge(self, this_parent, indent=0):
        penwidth = max(1, int(math.sqrt(self.descendant_count)))
        print(f'{indent*" "}"{this_parent.name}" -> "{self.name}" [penwidth={penwidth}];')
    def traverse_edge_defs(self, indent=0):
        """
        Write the DOT code that defines the incoming edges for this Node and
        all its descendants.
        
        Here and in traverse_node_defs(), 'indent' just helps with formatting
        when writing out the DOT code.
        """
        for kid in self.kids:
            kid.write_incoming_edge(self, indent=indent+4)
            kid.traverse_edge_defs(indent+4)


# Maintain a dictionary of Nodes so that we can find them by name.  Use
# paths relative to the root as keys- so the very first key is just '.'
nodes = {}
root_path = r'C:\Users\ajmek\OneDrive\Documents\GitHub\CMU-MS-DAS-Vis-S25\docs'
root_node = Node(None, '.')
nodes['.'] = root_node

# Walk the tree
for dirname, subdirs, files in os.walk(root_path):
    remove_excluded_subdirs(subdirs)
    rel_dir_path, rel_path = get_rel_paths(dirname, root_path)
    if rel_path in nodes:
        # This happens on the very first node
        dir_node = nodes[rel_path]
    else:
        assert rel_dir_path in nodes
        dir_node= Node(nodes[rel_dir_path], rel_path)
        nodes[rel_path] = dir_node

    # Add nodes for all the children of this dir
    for file in files:
        if show_this_leaf(file):
            full_path = os.path.join(dirname, file)
            ignore_this, rel_path = get_rel_paths(full_path, root_path)
            #print(f"Creating node for: {rel_path}")
            this_node = Node(dir_node, rel_path)
            nodes[rel_path] = this_node

# Calculate number of descendants for all nodes
for node in nodes:
    nodes[node].add_descendant_to_all_ancestors()

# Write out the Dot code
print("digraph {")
root_node.traverse_node_defs()
root_node.traverse_edge_defs()
print("}")

digraph {
"." [label=".", shape=ellipse, style=filled, fillcolor=lightgray];
    "assignment_ggplot.html" [label="assignm...", shape=rectangle, style=filled, fillcolor=lightgray];
    "assignment_ggplot_body.md" [label="assignm...", shape=rectangle, style=filled, fillcolor=lightgray];
    "assignment_ipywidgets.html" [label="assignm...", shape=rectangle, style=filled, fillcolor=lightgray];
    "assignment_ipywidgets_body.md" [label="assignm...", shape=rectangle, style=filled, fillcolor=lightgray];
    "assignment_maps.html" [label="assignm...", shape=rectangle, style=filled, fillcolor=lightgray];
    "assignment_maps_body.md" [label="assignm...", shape=rectangle, style=filled, fillcolor=lightgray];
    "assignment_matplotlib.html" [label="assignm...", shape=rectangle, style=filled, fillcolor=lightgray];
    "assignment_matplotlib_body.md" [label="assignm...", shape=rectangle, style=filled, fillcolor=lightgray];
    "assignment_node_edge_graphs.html" [label="assignm...", shape=rectangle